<h1>FantasyCalc.com Intro</h1>

Hey FDP readers! I'm Josh from <a target="_blank" href="https://www.fantasycalc.com">FantasyCalc.com</a>.  I've learned Python, R, Angular, SQL, AWS, and a whole lot more working on my fantasy football website over the past few years, so I'm excited to have the opportunity to collaborate by guest writing an article here.

FantasyCalc uses an algorithm to calculate player trade value from almost 1 million real fantasy football trades. In this article, we're going to compare those values to ADP to find over/undervalued players.

<h1>What do I need to know?</h1>

You should be familiar with lists, maps, and for loops from the first three intro to python articles. In this project, we'll cover APIs and JSON, and how we can combine multiple data sources for data analysis.

Application Program Interface (API) are how websites interact with backend databases and servers to get data. The APIs we're going to use return data in the JSON format - a common human readable response format consisting of strings, numbers, maps and lists.  We're going to use two in this article:

<ul>
<li><a target="_blank" href="https://api.fantasycalc.com/values/current?isDynasty=false&numQbs=1&numTeams=12&ppr=1">FantasyCalc.com's player value API</a>
</li>
<li>
<a target="_blank" href="https://fantasyfootballcalculator.com/api/v1/adp/standard?teams=12&year=2023">FantasyFootballCalculator's ADP API</a>
</li>
</ul>

If you click on the FantasyCalc link, you'll see what JSON data looks like.  Websites use HTML and CSS to turn JSON data into something that looks much prettier for human consumption.  For example, the FantasyCalc API is the exact same API that powers the <a target="_blank" href="https://www.fantasycalc.com/dynasty-rankings">rankings</a>.

<h1>Let's get started!</h1>

First, we'll want to import the data from the external APIs. We can do this using the requests library.  Let's import it and tell Python to expect JSON data.

In [5]:
import requests

trade_values = requests.get("https://api.fantasycalc.com/values/current?isDynasty=false&numQbs=1&numTeams=12&ppr=1").json()
adp = requests.get("https://fantasyfootballcalculator.com/api/v1/adp/standard?teams=12&year=2023").json()

trade_values[0]

{'player': {'id': 7569,
  'name': 'Justin Jefferson',
  'mflId': '14836',
  'sleeperId': '6794',
  'position': 'WR',
  'maybeBirthday': '1999-06-16',
  'maybeHeight': '73',
  'maybeWeight': 195,
  'maybeCollege': 'LSU',
  'maybeTeam': 'MIN',
  'maybeAge': 24.082668278253426,
  'maybeYoe': 3},
 'value': 10726,
 'overallRank': 1,
 'positionRank': 1,
 'trend30Day': -144,
 'redraftDynastyValueDifference': 0,
 'redraftDynastyValuePercDifference': 0,
 'redraftValue': 10726,
 'combinedValue': 21452,
 'maybeMovingStandardDeviation': 3,
 'maybeMovingStandardDeviationPerc': 0,
 'maybeMovingStandardDeviationAdjusted': 2,
 'displayTrend': False,
 'maybeOwner': None,
 'starter': False}

<h1>More on APIs</h1>

We use a combination of path and query parameters to tell the server exactly what data we want.  APIs are manually defined and will usually host documentation for how to use them. This is is too large of a topic to cover in this article, so <a target="_blank" href="https://www.abstractapi.com/api-glossary/path-parameters#:~:text=The%20path%20parameter%20defines%20the,client%20makes%20an%20API%20call">here's an article</a> if you'd like to learn a little more.

For now, we'll move on with our data imported.

<h1>Researching the data</h1>

We want to compare a player's ADP to their trade value. We're using two different sources with different response formats, so we'll need to figure out how we can compare them. Let's take a deeper look into the response JSON from both sites.

If you want, you can copy and paste the JSON responses into <a href="https://jsonviewer.stack.hu/" target="_blank">https://jsonviewer.stack.hu/</a> so it's easier to visualize the data.

Looking at <a href="https://api.fantasycalc.com/values/current?isDynasty=false&numQbs=1&numTeams=12&ppr=1" target="_blank">FantasyCalc.com's player value API</a>, we'll notice:

<ul>
  <li>The API returns a list of players</li>
  <li>Each player has a "overallRank"</li>
  <li>Each player has an object called "player" with an attribute "name"</li>
</ul>
Now, looking at <a href="https://fantasyfootballcalculator.com/api/v1/adp/standard?teams=12&year=2023" target="_blank">FantasyFootballCalculator's ADP API</a>, we'll notice:

<ul>
  <li>The response returns a Map with a key called "players"</li>
  <li>Each player has a "name" and "adp" key</li>
</ul>
So for each player, we'll want to compare FantasyCalc's "overallRank" to FFC's "adp".

<h1>Reformatting the data</h1>
Now that we know which values on the JSON response we want to compare, we can reformat each source into the same format. We'll want to be able to lookup a player's ADP or trade value from their name, so we'll use the Map data structure.

In [7]:
adp_map = {}
for player in adp['players']:
    adp_map[player['name']] = player['adp']

trade_values_map = {}
for player in trade_values:
    trade_values_map[player['player']['name']] = player['overallRank']

print('ADP map: ', adp_map)
print('Trade value map: ', trade_values_map)

ADP map:  {'Christian McCaffrey': 1.3, 'Austin Ekeler': 2.0, 'Jonathan Taylor': 2.7, 'Justin Jefferson': 3.7, 'Saquon Barkley': 3.9, "Ja'Marr Chase": 6.4, 'Kenneth Walker III': 6.8, 'Nick Chubb': 7.8, 'Josh Jacobs': 7.9, 'Travis Kelce': 9.5, 'Derrick Henry': 9.7, 'Tony Pollard': 11.9, 'Travis Etienne': 12.3, 'Tyreek Hill': 12.9, 'Cooper Kupp': 13.8, 'Breece Hall': 15.6, 'Davante Adams': 16.3, 'Bijan Robinson': 17.4, 'CeeDee Lamb': 17.7, 'Stefon Diggs': 18.1, 'Najee Harris': 19.0, 'A.J. Brown': 19.2, 'George Kittle': 23.8, 'Jaylen Waddle': 23.8, 'Mark Andrews': 24.5, 'Dalvin Cook': 24.7, 'Rhamondre Stevenson': 25.4, 'Amon-Ra St. Brown': 26.3, 'Garrett Wilson': 28.1, 'Tee Higgins': 29.0, 'J.K. Dobbins': 29.6, 'Josh Allen': 29.8, 'Patrick Mahomes': 30.8, 'Chris Olave': 31.6, 'DeVonta Smith': 31.7, 'Aaron Jones': 33.1, 'Calvin Ridley': 34.3, 'Deebo Samuel': 37.0, 'Cam Akers': 38.6, 'D.K. Metcalf': 38.9, 'Amari Cooper': 39.5, 'T.J. Hockenson': 39.9, 'Jalen Hurts': 41.0, 'Miles Sanders': 41.

<h1>Comparing the data</h1>

Now we have both data sources in the same format, we can easily compare the two.  Let's loop over the trades values, and use the name key to get that player's trade value, and print the result.

Note the trade values might have some different players than ADP.  If we have a player in the trade values that does not have ADP data, we'll just skip them.

In [8]:
for name, player_trade_value in trade_values_map.items():
    player_adp = adp_map.get(name)

    # Skip players without ADP data
    if player_adp is None:
        continue

    diff = player_trade_value - player_adp

    print(name, player_trade_value, player_adp, diff)

Justin Jefferson 1 3.7 -2.7
Christian McCaffrey 2 1.3 0.7
Ja'Marr Chase 3 6.4 -3.4000000000000004
Bijan Robinson 4 17.4 -13.399999999999999
Cooper Kupp 5 13.8 -8.8
Austin Ekeler 6 2.0 4.0
Travis Kelce 7 9.5 -2.5
A.J. Brown 8 19.2 -11.2
Jonathan Taylor 9 2.7 6.3
CeeDee Lamb 10 17.7 -7.699999999999999
Nick Chubb 11 7.8 3.2
Patrick Mahomes 12 30.8 -18.8
Saquon Barkley 13 3.9 9.1
Stefon Diggs 14 18.1 -4.100000000000001
Tyreek Hill 15 12.9 2.0999999999999996
Derrick Henry 16 9.7 6.300000000000001
Josh Jacobs 17 7.9 9.1
Garrett Wilson 18 28.1 -10.100000000000001
Jaylen Waddle 19 23.8 -4.800000000000001
Josh Allen 20 29.8 -9.8
Tony Pollard 21 11.9 9.1
Davante Adams 22 16.3 5.699999999999999
Amon-Ra St. Brown 23 26.3 -3.3000000000000007
Breece Hall 24 15.6 8.4
Rhamondre Stevenson 25 25.4 -0.3999999999999986
Jalen Hurts 26 41.0 -15.0
Chris Olave 27 31.6 -4.600000000000001
Travis Etienne 28 12.3 15.7
Jahmyr Gibbs 30 112.6 -82.6
Tee Higgins 31 29.0 2.0
Mark Andrews 33 24.5 8.5
Najee Harris 34 19.

<h1>Formatting our results</h1>

Now we have the comparison data between ADP and trade value!  We'll want to sort it though, so the results show our top over/undervalued players.

Instead of printing each row, let's add them to a list and sort the list at the end.

In [12]:
diffs = []
for name, player_trade_value in trade_values_map.items():
    player_adp = adp_map.get(name)

    # Skip players without ADP data
    if player_adp is None:
        continue

    diff = player_trade_value - player_adp

    diffs.append([name, player_trade_value, player_adp, diff])


# Use a lambda to sort by the 'diff' we defined above.
# d[3] is sorting by the 4th element of the list since lists are indexed started from zero.
sorted_diffs = sorted(diffs, key=lambda d: d[3])
for diff in sorted_diffs:
    print(diff)

['Jahmyr Gibbs', 30, 112.6, -82.6]
['Jaxon Smith-Njigba', 63, 136.4, -73.4]
['Anthony Richardson', 76, 147.7, -71.69999999999999]
['Devon Achane', 92, 160.8, -68.80000000000001]
['Zach Charbonnet', 82, 146.1, -64.1]
['Zay Flowers', 88, 150.2, -62.19999999999999]
['Jordan Addison', 86, 146.9, -60.900000000000006]
['Quentin Johnston', 84, 144.4, -60.400000000000006]
['Kyler Murray', 89, 142.2, -53.19999999999999]
['Trevor Lawrence', 40, 70.3, -30.299999999999997]
['Diontae Johnson', 66, 94.7, -28.700000000000003]
['Dalton Kincaid', 114, 140.8, -26.80000000000001]
['Skyy Moore', 124, 150.6, -26.599999999999994]
['Samaje Perine', 94, 119.1, -25.099999999999994]
['Jordan Love', 136, 158.9, -22.900000000000006]
['Jerick McKinnon', 102, 124.7, -22.700000000000003]
['Justin Herbert', 43, 65.6, -22.599999999999994]
['Alvin Kamara', 49, 71.5, -22.5]
['Lamar Jackson', 39, 60.0, -21.0]
['Javonte Williams', 52, 73.0, -21.0]
['Deshaun Watson', 68, 87.5, -19.5]
['Alexander Mattison', 74, 93.3, -19.29

We can now use `pandas`, Python's data manipulation library, to format our data as a table for better readability.

In [16]:
import pandas as pd
df = pd.DataFrame(sorted_diffs, columns=['name', 'trade_value_rank', 'adp', 'tv - adp'])
with pd.option_context('display.max_rows', None):
  display(df)

,name,trade_value_rank,adp,tv - adp
0,Jahmyr Gibbs,30,112.6,-82.6
1,Jaxon Smith-Njigba,63,136.4,-73.4
2,Anthony Richardson,76,147.7,-71.7
3,Devon Achane,92,160.8,-68.8
4,Zach Charbonnet,82,146.1,-64.1
5,Zay Flowers,88,150.2,-62.2
6,Jordan Addison,86,146.9,-60.9
7,Quentin Johnston,84,144.4,-60.4
8,Kyler Murray,89,142.2,-53.2
9,Trevor Lawrence,40,70.3,-30.3


<h1> Challenge yourself</h1>

<ul>
<li>The trade_value and adp diffs include a lot of decimals. Can you print this in a prettier format?  Example: 8.2000000003 -> 8</li>
<li>Can you figure out how to include players whose name's don't exactly match between the two sources? Example: DK Metcalf</li>
</ul>

# Concluding thoughts

Fetching, reformatting, and comparing multiple data sources is a very common programming task.  I'm excited I got the opportunity to teach an example using fantasy football data.  I've spent thousands of hours learning Python, web development, infrastructure, and more on my own fantasy football projects, and have got plenty of help from the fantasy community, so I'm happy to take this opportunity to collaborate to pay it forward.